Quran Chatbot

In [25]:
import os
import sqlite3
import openai
import gradio as gr
from dotenv import load_dotenv

In [26]:
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")

In [27]:
def search_quran(keyword):
    conn = sqlite3.connect("db/quran.db")
    cursor = conn.cursor()
    query = """
    SELECT surahId, ayatNumber, arabicText, urduTranslation 
    FROM quran 
    WHERE arabicText LIKE ? OR urduTranslation LIKE ?
    LIMIT 5
    """
    pattern = f"%{keyword}%"
    cursor.execute(query, (pattern, pattern))
    results = cursor.fetchall()
    conn.close()
    return results


In [28]:
class QuranChatbot:

    def __init__(self):
        self.openai = openai.OpenAI(api_key=openai_api_key)

    
    def system_prompt(self):
        return (
            "Aap ek Islami scholar hain. User Urdu mein Quran se related sawal poochta hai. "
            "Aap relevant ayatein aur tafsir Urdu mein samjhate hain. "
            "Agar koi sawal mushkil ho ya jawab nahi milta, use record karen."
        )

    def chat(self, message, history):
        # Step 1: Extract keywords via OpenAI
        keyword_extraction_prompt = (
            f"Extract 1-2 important keywords (in Urdu) from this Quran-related question "
            f"to help look up related ayats:\n\n'{message}'\n\nJust return the keywords as a comma-separated string."
        )

        keyword_response = self.openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You extract keywords from Urdu/Arabic Quran-related queries."},
                {"role": "user", "content": keyword_extraction_prompt}
            ]
        )

        keyword_text = keyword_response.choices[0].message.content.strip()
        extracted_keywords = [kw.strip() for kw in keyword_text.split(",") if kw.strip()]

        # Step 2: Search Quran DB with keywords
        results = []
        for kw in extracted_keywords:
            matches = search_quran(kw)
            if matches:
                results.extend(matches)

        results = list({(s, a, ar, ur) for s, a, ar, ur in results})

        if not results:
            return "Maaf kijiye, Quran mein is sawal se mutaliq koi ayat nahi mili."

        # Step 3: Format ayats for OpenAI input
        context = "Yeh Quran ki kuch ayatein hain jo aapke sawal se mutaliq ho sakti hain:\n\n"
        for surahId, ayatNumber, arabicText, urduTranslation in results[:5]:
            context += f"Surah {surahId}, Ayat {ayatNumber}:\n"
            context += f"Arabic: {arabicText}\n"
            context += f"Urdu: {urduTranslation}\n\n"

        prompt = context + f"Aur ab user ka sawal hai:\n\n{message}\n\nIn ayaton ki roshni mein tafsir samjhayen."

        # Step 4: Chat with OpenAI
        messages = [{"role": "system", "content": self.system_prompt()}]
        for msg in history:
            messages.append({"role": msg["role"], "content": msg["content"]})
            messages.append({"role": "user", "content": prompt})

        done = False
        while not done:
            response = self.openai.chat.completions.create(
                model="gpt-4o",
                messages=messages,
            )
            done = True

        return response.choices[0].message.content


In [29]:
quran_bot = QuranChatbot()

gr.ChatInterface(fn=quran_bot.chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
